# Analysis
The most naive way to calculate the forces in a system of particles is called the pair-wise (PW) algorithm. The interactions between each particle pair are calculated directly, the are $\frac{N(N+1)}{2}$ pairs of particles for an N-particle system. The PW algorithm therefore has $O(N^2)$ time complexity, this is not scalable for large numbers of particles, so I seek to investigate different algorithms and how their respective performance and accuracy compare.

## Barnes-Hut Algorithm
The Barnes-Hut (BH) Algorithm divides space into quadrants, to form a quadtree of particles. For the PW algorithm, the interaction list length for each particle is $O(N)$, updating every particle is therefore $O(N^2)$ complexity. Using the quadtree structure, the tree can be iterated through recursively, making approximations at each level. The approximation reduces the interaction list length to $O(\log{N})$ resulting in overall $O(N \log{N})$ complexity.

The reduction in computation time comes at the cost of accuracy, controlled by a parameter $\theta$. For a quadrant with width $d$, that is a distance $r$ away from a point, the ratio $\frac{d}{r}$ is considered. If $\frac{d}{r} \lt \theta$ the quadrant is sufficiently far away and we can approximate the interaction with the centre of charge, rather than each individual particle. If $\frac{d}{r} \ge \theta$, the algorithm is repeated recursively for each sub-quadrant.

The actual time complexity depends on the value of $\theta$, for example when $\theta=0$ the approximation is never made and interaction list contains all the particles. This is simply just the PW algorithm with $O(N^2)$ complexity. However, as $\theta \to \infty$, the approximation is always taken at the root of the tree meaning the interaction list always contains only one item, the performance in this case would be $O(N)$

### Filling the Quadtree

The Quadtree class has two states, whether it has been *divided* or not. A divided quadtree contains 4 children quadtrees and no particles, an undivided quadtree contains no children and a list of particles. The first particle is added to the quadtree particle list, when the second particle is added, the quadtree must divide and the particles are added to the children quadtrees. This process of either appending the particle to the list, or dividing and passing the particle to th children is continued until there are no more particles to add or the maximum depth is reached.

<img src="diagrams/tree_fill.jpeg" width=600px>

### Finding the Interaction List

The interaction list is the list of charges that a particle must interact with. To find the interaction list for a particle, the ratio $\frac{r}{d}$ is calcualted for the root quadtree, if this ratio is $\lt\theta$, the quadrant is far enough away from the particle that we can approximate and only append the quadrant's centre of charge, instead of every particle that the quadrant contains. However, if the ratio $\frac{r}{d} \ge \theta$, the particle is too close to the quadrant to make this approximation and the interaction list is the union of the interaction lists of the sub-quadrants.

Once the tree has been traversed, the interaction list contains particles and centres of charges for which the interaction is calculated directly. The diagram below shows how the interaction list is found, with the final interaction list at the end.

<img src="diagrams/algorithm.jpeg" width="600px">

The adaptive quadtree structure, and interaction list for a point at the centre are displayed below

<img id="BH_squares" src="animations/BH.gif" width="600px">
<img src="animations/BH_clusters.gif" width="600px">


## Fast Multipole Method (FMM)

The fast multipole method is another algorithm to calculate interactions between particles with complexity $O(N)$. FMM has some similarities to BH in the grid structure, however instead of calculating centres of charges, multipole expansions are calculated with the number of terms defining the accuracy of the algorithm.

### Calculating Multipoles

For a 2D simulation, the system can be viewed as particles lying in a plane in 3D, or a universe that is only 2D. The interactions between particles will depend on whichever viewpoint is taken. For FMM, I will consider a 2D universe with force $F(r) \propto \frac{1}{r}$ and potential $\Phi(r) \propto \ln(r)$. To make the mathematics easier, complex numbers are used as position coordinates, with their real/imaginary components corresponding to the $x/y$ coordinates. Expressing a complex number in exponential form makes the potential calculation simple. 
$$z = r e^{i\theta}$$
$$\ln(z) = \ln(r) + i\theta$$
The potential is then simply just the real part of $\ln(r)$ where r is the difference between the locations of two particles.

For a cluster of charges located at the points $\{z_i, i=0, 1, 2, ...\}$, the potential induced by the charges at a location $z$ is given by the multipole expansion: $$\phi(z) = Q \ln(z) + \sum_{k=1}^\infty \frac{a_k}{z^k}$$
For total charge $Q$ and expansion coefficients $a_k$

$$
Q = \sum_{i=1}^m q_i\hspace{1cm} a_k = \sum_{i=1}^m \frac{-q_i z_i^k}{k}
$$

### Multipole Errors

The error in the multipole expansion depends on the number of terms $p$. If the cluster of particles has *radius* $r$ the absolute error in the potential for any $p \ge 1$ is constrained by:
$$\begin{align*}
\bigg|\phi(z) - Q \ln(z) - \sum_{i=1}^p \frac{a_k}{z^k}\bigg| &\le \frac{1}{p+1} \alpha \bigg| \frac{r}{z} \bigg|^{p+1}\\
&\le \frac{A}{p-1} \frac{1}{c-1} \left(\frac{1}{c}\right)^p
\end{align*}
$$

$$c = \bigg| \frac{z}{r} \bigg|, \hspace{.5cm}
A = \sum_{i=1}^m |q_i|, \hspace{.5cm}
\alpha = \frac{Ac}{c-1}$$

Therefore for a given tolerence $\epsilon$ in the potential, use $p = -\log_c{\epsilon}$

### Partitioning Space

Considering the case of uniform particle distribution, the space can be split into grids of different layers, each layer containing cells with half the size of the layer above. More sophisticated methods can be used, for example an adaptive tree structure would be better suited for non-uniform particle distributions. For the simpler case, the data structure consists of a list of $2^l \times 2^l \times p$ arrays where $l$ is the depth of the layer, and $p$ is the number of terms in the expansion to store.

The terminology I will frequently use:

**Neighbour** : cell-1 is a neighbour of a cell-2 if there are no cells inbetween cell-1 and cell-2, and the cells are on the same layer. This includes diagonals

**Well-Seperated** : The well-seperated cells of cell-1 are the children of the cells that are siblings of the parent of cell-1, and that are also not neighbours of cell-1. A close analogy for this is the *cousins* of cell-1

<img src="diagrams/structure.jpeg" width=600>

### Upward Pass

The first step of the algorithm is the upward pass. Starting at the leaf layer, each multipole expansion is converted into another multipole expansion located at the centre of the parent cell. This is repeated for every cell in every layer, moving upwards. The Multipole-Multipole conversion will be called M2M

The conversion of the expansion coefficients going from $z_C$ to $z_M$ is given by the following transformation:
$$\phi(z) = b_0 \ln(z-z_M) + \sum_{l=1}^p \frac{b_l}{(z-z_M)^l} + O(\epsilon)\\
b_0 = a_0, \hspace{0.5cm} b_l = -\frac{a_0(z_C-z_M)^l}{l} + \sum_{k=1}^l a_k(z_C - z_M)^{l-k} {l-1 \choose k-1}$$

<img src="diagrams/upward.jpeg" width=600>

### Downward Pass

The downward pass does two things at each layer, a multipole-to-local expansion conversion (M2L) and local-to-local expansion conversion (L2L).

M2L considers all the well-seperated cells of a cell and converts their multipole expansions into local expansions located at the centre of the cell. This is done for every cell in the layer. The local expansion coefficients going from $z_M$ to $z_L$ are:
$$\phi(z) = \sum_{l=0}^p c_l(z-z_L)^l + O(\epsilon)$$
$$c_0 = b_0 \ln(z_L - z_M) + \sum_{k=1}^p \frac{(-1)^k b_k}{(z_M-z_L)^k}, \\
c_l = -\frac{b_0}{l(z_M - z_L)^l} + \frac{1}{(z_M - z_L)^l} \sum_{k=1}^p \frac{(-1)^k b_k}{(z_M-z_L)^k} {l+k-1 \choose k-1}$$
L2L then converts the local expansion of a cell into more local expansions located at the centres of the the cell's children. The new expansion coefficients going from $z_L$ to $z_T$ are:
$$\phi(z) = \sum_{l=0}^p d_l (z-z_T)^l\\
d_l = \sum_{k=1}^p c_k {k \choose l} (z_T - z_L)^{k-l}$$


<img src="diagrams/downward.jpeg" width=600>

As shown in the diagram above, performing M2L at each layer means that the multipole expansions of each particle are taken into consideration. After the downward pass the algorithm is finished and the potentials or forces can be calculated. As well as the multipole expansion, the interactions between particles in neighbouring cells needs to be computed directly.

### Forces and Potentials

The potential is given by the expansion $\phi(z) = \sum_{l=0}^p d_l(z-z_T)^l$ where $d_l$ are the local expansion coefficients
To get the force, the gradient of the expansion is taken, using the Cauchy-Riemann equations. For the potential $\phi(z) = \Re(w(z))$, the gradient is $$\nabla \phi = (f_x, f_y) = \left(\Re(\frac{\partial w}{\partial z}), -\Im(\frac{\partial w}{\partial z})\right)$$
$$
\begin{align*}
F &= -\nabla \phi(z) \\
&= \sum_{l=1}^p l d_l(z-z_T)^{l-1}\\

\end{align*}
$$
Finally our force components are $$F_x = \Re(F) \hspace{0.5cm} F_y = -Im(F)$$

### FMM Complexity

1. Multipole Expansions for each particle - $O(Np)$
2. M2M - $O(Np^2)$
3. M2L - $\le O(27Np^2)$
4. L2L - $O(Np^2)$
5. Evaluate local expansions for each particle - $O(Np)$
6. Direct neighbour interactions - $O(\frac{9}{2}Nk_n)$ ($k_n =$ particles per cell) 
7. Combining far-field expansions and direct interactions - $O(N)$

Overall the algorithm has $O(N)$ time complexity

# Implementation

My aim was to write code that made switching between algorithms as easy as possible, I created an algorithm class that each algorithm could inherit from. Combined with a universe class and a particle class, the system can simulate and animate the interactions for any algorithm.

### Particle Class

The particle class contains the necessary properties for an interacting particle - mass, charge, position, velocity and potential. There is also the index method, this returns the index of the cell that the particle would lie in if it were in a quadtree like structure. The particle position and velocity are stored as complex numbers and if the interaction is gravitational, the charge is equal to the mass.
```python
class Particle:
    def __init__(self, charge: float, mass: float, 
                 centre: complex, velocity: complex) -> None:
    self.charge = charge
    self.mass = mass
    self.centre = centre
    self.velocity = velocity
    self.potential = 0

    def index(self, level: int) -> Index:
        i = int(self.centre.imag * 2 ** level)
        j = int(self.centre.real * 2 ** level)

        return Index(i, j, level)
```

### Index Class
```py
 class Algorithm(ABC):
    def __init__(self):
        self.calculate_potential = None

    @abstractmethod
    def calculate_accelerations(self, particles: list[Particle]) -> \
            NDArray[complex]:

        return NDArray[complex]

    @abstractmethod
    def animate(self, fig: plt.Figure, ax: plt.Axes, scatter: plt.Subplot,
                **kwargs) -> None:
        return
```